<img src="image.jpg">

 # PyPRT - Building Modeling Optimization

This notebook is used to show how the Python bindings for PRT can be employed in optimizing the rule attributes of a generated model to achive predefined performance goals.

TODO:
    - change "total report for optimisation" to "building greenery (trees and ground greenery)" or something like that in Ioanna's rule
    - find a way to put a limit to the minimum thickness of the building walls
    - put different weight (<10) for the number of trees: total report for optimization = green area + 10* nber of trees in order to have a better results for the step 1 optimization
    
    - research for other python library that has optimiaztion algorithms with possibility to define input attributes step size

#### Imports and paths-related functions

In [32]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), "..", "src"))
from utility import visualize_PRT_results, combine_reports

SDK_PATH = os.path.join(os.getcwd(), "..", "install", "bin")
sys.path.append(SDK_PATH)

import pyprt

from scipy import optimize
import numpy as np

CS_FOLDER = os.getcwd()
def asset_file(filename):
    return os.path.join(CS_FOLDER, filename)

from IPython.display import IFrame
from arcgis import GIS

In [33]:
my_gis = GIS(username="CLechot_zurich")

Enter password: ········


In [34]:
print("\nInitializing PRT.")
pyprt.initialize_prt(SDK_PATH)

if not pyprt.is_prt_initialized():
    raise Exception("PRT is not initialized")


Initializing PRT.


## Optimization Problem Definition

In this notebook, we are using a CGA rule that allows us to generate modern and sustainable buildings. The architecture of these buildings is focused on the green aspect of the building. Therefore, in addition to be modern and esthetical, the generated building answers also to the actual requirements of the city which are to encourage the integration of greenery on the buildings.

<img src="green_building_parameters.png" width="1000" height="100">

In [35]:
rpk_green = asset_file("envelope2409.rpk") #envelope1806.rpk
attrs_green = {'ruleFile' : "rules/typology/envelope2409.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True}

shape_geo = asset_file("parcelN.obj")

### Optimization with envelope.rpk

Step 1 : optimize the green area based on the lot coverage parameter

### The variables

In [36]:
goal_str_green = "Greenery Area" + "_sum"

input_attr_green = ["lot_coverage_parameter"]
x0_green = [float(50)]

In [37]:
m = pyprt.ModelGenerator(shape_geo)
model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-09-30 15:39:55] [debug] trying to read initial shape geometry from file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5CparcelN.obj

[PRT] [2019-09-30 15:39:55] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-30 15:39:55] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/unified_landscape/plan4.obj" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/unified_landscape/plan4.obj" type="str"/>
	<attribute key="/green_optimization_3/rules/typology/ground_floor_plan.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/rules/typology/ground_floor_plan.cga" type="str"/>
	<attribute key="/ESRI.lib/rules/Plan

In [7]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [8]:
bounds_green = [(0,100)]

In [9]:
-fct_green(x0_green)

2168.6307416254404

In [10]:
import time
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, seed = 666) # dual_annealing

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

The optimization lasted: 28.710047960281372


     fun: -2196.7870170993974
 message: 'Optimization terminated successfully.'
    nfev: 32
     nit: 1
 success: True
       x: array([82.26263821])

### Results Visualization

In [12]:
export_file_name = "Demo_SLPKModel1"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

attrs_green[input_attr_green[0]] = res_green.x[0]
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-09-30 14:19:22] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-30 14:19:22] [debug] resolve map = <ResolveMap>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Distributor.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Distributor.cga" type="str"/>
	<attribute key="assets/vertical_greenery/6.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/vertical_greenery/6.PNG" type="str"/>
	<attribute key="rules/typology/landscape.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/rules/typology/landscape.cga" type="str"/>
	<attribute key="assets/vertical_greenery/4

[PRT] [2019-09-30 14:19:24] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:19:24] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:19:24] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:19:24] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:19:24] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:19:24] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:19:24] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:19:25] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:19:25] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:19:25] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:19:25] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:19:25] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:19:25] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:19:25] [debug] width 489, heig

In [13]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
if item is None:
    # first delete the existing one if any and add
    my_gis.content.delete_items([item])
    item = my_gis.content.add({'type':'Scene Package'}, asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
published_item = item.publish()

In [14]:
source = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id
IFrame(src=source, width=900, height=600)

-------------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------

Step 2 : optimize the total green area reported including the balconies trees. This will be optimized taking into account several parameters:
- the lot coverage
- the height of the first tier

### The variables

In [15]:
goal_str_green = "Greenery Area" + "_sum"

input_attr_green = ["lot_coverage_parameter", "height_first_tier"]
x0_green = [float(80), float(9)]

In [16]:
attrs_green = {'ruleFile' : "rules/typology/envelope2409.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True}

model2 = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-09-30 14:21:13] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-30 14:21:13] [debug] resolve map = <ResolveMap>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Distributor.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Distributor.cga" type="str"/>
	<attribute key="assets/vertical_greenery/6.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/vertical_greenery/6.PNG" type="str"/>
	<attribute key="rules/typology/landscape.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/rules/typology/landscape.cga" type="str"/>
	<attribute key="assets/vertical_greenery/4

In [17]:
visualize_PRT_results(model2)


Number of generated geometries (= nber of initial shapes):
1

Initial Shape Index: 0

Report of the generated model:
{'Greenery Area_avg': 3.699304017412569, 'Greenery Area_max': 408.2737197150759, 'Greenery Area_min': 0.4895963195850646, 'Greenery Area_n': 574.0, 'Greenery Area_sum': 2123.4005059948145, 'Number of trees_avg': 1.0, 'Number of trees_max': 1.0, 'Number of trees_min': 1.0, 'Number of trees_n': 114.0, 'Number of trees_sum': 114.0, 'green area_avg': 3.3769576217278554, 'green area_max': 408.2737197150759, 'green area_min': 0.4895963195850646, 'green area_n': 460.0, 'green area_sum': 1553.4005059948136}



In [18]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    attrs_green[input_attr_green[1]] = x[1]
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [19]:
bounds_green = [(0,100),(8,13)]

In [20]:
-fct_green(x0_green)

2112.7263646109823

In [21]:
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, seed = 666) #dual_annealing

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

The optimization lasted: 160.53169918060303


     fun: -2221.57314995954
 message: 'Optimization terminated successfully.'
    nfev: 183
     nit: 5
 success: True
       x: array([99.08078256, 12.79276757])

In [22]:
attrs_green[input_attr_green[0]] = res_green.x[0]
attrs_green[input_attr_green[1]] = res_green.x[1]
green_model2 = m.generate_model(attrs_green)

if green_model2:
    visualize_PRT_results(green_model2)


Number of generated geometries (= nber of initial shapes):
1

Initial Shape Index: 0

Report of the generated model:
{'Greenery Area_avg': 3.006188294938484, 'Greenery Area_max': 196.2969930996951, 'Greenery Area_min': 0.4895976423336479, 'Greenery Area_n': 739.0, 'Greenery Area_sum': 2221.57314995954, 'Number of trees_avg': 1.0, 'Number of trees_max': 1.0, 'Number of trees_min': 1.0, 'Number of trees_n': 138.0, 'Number of trees_sum': 138.0, 'green area_avg': 2.54837462555664, 'green area_max': 196.2969930996951, 'green area_min': 0.4895976423336479, 'green area_n': 601.0, 'green area_sum': 1531.5731499595408}



### Results Visualization

In [23]:
export_file_name = "Demo_SLPKModel2"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

#attrs_green[input_attr_green[0]] = res_green.x[0]
#attrs_green[input_attr_green[1]] = res_green.x[1]
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-09-30 14:24:23] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-30 14:24:23] [debug] resolve map = <ResolveMap>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Distributor.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Distributor.cga" type="str"/>
	<attribute key="assets/vertical_greenery/6.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/vertical_greenery/6.PNG" type="str"/>
	<attribute key="rules/typology/landscape.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/rules/typology/landscape.cga" type="str"/>
	<attribute key="assets/vertical_greenery/4

[PRT] [2019-09-30 14:24:26] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:24:26] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:24:27] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 14:24:27] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 14:24:27] [debug] width 850, heig

In [24]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
if item is None:
    # first delete the existing one if any and add
    my_gis.content.delete_items([item])
    item = my_gis.content.add({'type':'Scene Package'}, asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
published_item = item.publish()

In [25]:
source2 = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id
IFrame(src=source2, width=900, height=600)

-------------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------

Step 3 : optimize the total green area reported including the balconies trees. This will be optimized taking into account several parameters:
- the shape of the building (1 = orthogonal, 2 = L shape, 3 = U shape, 4 = O shapes)
- the lot coverage
- the height of the first tier

### The variables

In [38]:
goal_str_green = "Greenery Area" + "_sum"

input_attr_green = ["lot_coverage_parameter", "height_first_tier", "shape_of_building"]
x0_green = [float(50), float(9), float(int(2))]

In [39]:
attrs_green = {'ruleFile' : "rules/typology/envelope2409.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True}

model3 = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-09-30 15:40:05] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-30 15:40:05] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/unified_landscape/plan4.obj" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/unified_landscape/plan4.obj" type="str"/>
	<attribute key="/green_optimization_3/rules/typology/ground_floor_plan.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/rules/typology/ground_floor_plan.cga" type="str"/>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Loader.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Loader.cga

In [40]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    attrs_green[input_attr_green[1]] = x[1]
    attrs_green[input_attr_green[2]] = float(round(x[2]))
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [41]:
bounds_green = [(0,100),(8,13),(1,4)]

In [42]:
-fct_green(x0_green)

2566.649574933355

In [43]:
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, seed = 666) #dual_annealing

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

[PRT] [2019-09-30 15:51:51] [warning] Rule 'Default$volume$Slab_texture()V' InitialShape '<?>': illegal component geometry, no shape created.
[PRT] [2019-09-30 15:51:51] [warning] Rule 'Default$volume$Slab_texture()V' InitialShape '<?>': illegal component geometry, no shape created.
[PRT] [2019-09-30 15:56:05] [warning] Rule 'Default$volume$Slab_texture()V' InitialShape '<?>': illegal component geometry, no shape created.
[PRT] [2019-09-30 15:56:05] [warning] Rule 'Default$volume$Slab_texture()V' InitialShape '<?>': illegal component geometry, no shape created.
The optimization lasted: 1173.8636429309845


     fun: -4230.006530381527
 message: 'Optimization terminated successfully.'
    nfev: 1309
     nit: 28
 success: True
       x: array([33.13302682, 12.80920399,  3.51896445])

In [44]:
attrs_green[input_attr_green[0]] = res_green.x[0]
attrs_green[input_attr_green[1]] = res_green.x[1]
attrs_green[input_attr_green[2]] = float(round(res_green.x[2]))
green_model3 = m.generate_model(attrs_green)

if green_model3:
    visualize_PRT_results(green_model3)


Number of generated geometries (= nber of initial shapes):
1

Initial Shape Index: 0

Report of the generated model:
{'Greenery Area_avg': 3.112587586741374, 'Greenery Area_max': 456.3668332213981, 'Greenery Area_min': 0.4895963195850646, 'Greenery Area_n': 1359.0, 'Greenery Area_sum': 4230.006530381527, 'Number of trees_avg': 1.0, 'Number of trees_max': 1.0, 'Number of trees_min': 1.0, 'Number of trees_n': 264.0, 'Number of trees_sum': 264.0, 'green area_avg': 2.6575402103940897, 'green area_max': 456.3668332213981, 'green area_min': 0.4895963195850646, 'green area_n': 1095.0, 'green area_sum': 2910.006530381528}



### Results Visualization

In [45]:
export_file_name = "Demo_SLPKModel3"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

#attrs_green[input_attr_green[0]] = res_green.x[0]
#attrs_green[input_attr_green[1]] = res_green.x[1]
#attrs_green[input_attr_green[2]] = res_green.x[2]
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-09-30 16:01:02] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope2409.rpk

[PRT] [2019-09-30 16:01:02] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/unified_landscape/plan4.obj" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/assets/unified_landscape/plan4.obj" type="str"/>
	<attribute key="/green_optimization_3/rules/typology/ground_floor_plan.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/rules/typology/ground_floor_plan.cga" type="str"/>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Loader.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope2409.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Loader.cga

[PRT] [2019-09-30 16:01:04] [debug] width 32, height 32, rasterCount 3
[PRT] [2019-09-30 16:01:04] [debug] width 32, height 32, rasterCount 3
[PRT] [2019-09-30 16:01:04] [debug] width 32, height 32, rasterCount 3
[PRT] [2019-09-30 16:01:04] [debug] width 64, height 32, rasterCount 3
[PRT] [2019-09-30 16:01:04] [debug] width 64, height 32, rasterCount 3
[PRT] [2019-09-30 16:01:04] [debug] width 64, height 32, rasterCount 3
[PRT] [2019-09-30 16:01:05] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 16:01:05] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 16:01:05] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 16:01:05] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 16:01:05] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 16:01:05] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-09-30 16:01:05] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-09-30 16:01:05] [debug] width 850, height 850, rast

In [46]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
if item is None:
    # first delete the existing one if any and add
    my_gis.content.delete_items([item])
    item = my_gis.content.add({'type':'Scene Package'}, asset_file("..\output\%s.slpk" % export_file_name), folder='packages')
published_item = item.publish()

In [47]:
source3 = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id
IFrame(src=source3, width=900, height=600)

In [48]:
print("\nShutdown PRT.")
pyprt.shutdown_prt()


Shutdown PRT.
